# BERT Sentiment Analysis Model Server
The model used here, was trained with the concept of transfer learning  i.e. taking huggingface transformers pretrained BERT model and further training it on a custom dataset of reviews. this yields a sentiment analysis model based on the prior knowledge of BERT. 
The model server is given a list of texts and outputs a list of labels corresponding to its prediction.
The labels express the sentiment of the writer towards the topic of the text:
0 for negative sentiment, 1 for neutral and 2 for positive.

The model file (~430 MB), can be downloaded to your local environment from: https://iguazio-sample-data.s3.amazonaws.com/models/model.pt

In [1]:
from pathlib import Path
from mlrun import import_function

In [2]:
MODEL_PATH = Path("model") / "model.pt"
MODEL_URL = "https://iguazio-sample-data.s3.amazonaws.com/models/model.pt"

In [3]:
def download_file(url, save_to):
    import requests
    from IPython.display import clear_output

    url_file = url.split("/")[-1]
    save_to = Path(save_to)

    save_to_file = save_to if "." in save_to.name else (save_to / url_file)

    if not save_to.parent.exists():
        save_to.parent.mkdir(parents=True, exist_ok=True)

    with requests.get(url, stream=True) as response:
        response.raise_for_status()
        content_length = float(response.headers["content-length"])
        downloaded = 0

        with open(save_to_file, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                clear_output(wait=True)
                downloaded += len(chunk)
                f.write(chunk)
                done = int(50 * downloaded / content_length)
                print(f"Downloading: {url}")
                print(f"[{'=' * int(done)}{' ' * (50-int(done))}]")
        print(f"Done downloading: {save_to_file.resolve()}")

In [4]:
if not MODEL_PATH.exists():
    download_file(MODEL_URL, MODEL_PATH)

Downloading: https://iguazio-sample-data.s3.amazonaws.com/models/model.pt
[==================================================] 100%
Done downloading: /home/michaell/projects/functions/sentiment_analysis_serving/model/model.pt


In [5]:
from sentiment_analysis_serving import *

fn = import_function("function.yaml")
fn.add_model("bert", model_path=str(MODEL_PATH.resolve()), class_name="SentimentClassifierServing")
server = fn.to_mock_server()

RuntimeError: Error(s) in loading state_dict for BertSentimentClassifier:
	Missing key(s) in state_dict: "bert.embeddings.position_ids". 

## Example 1 - Positive Sentiment

In [9]:
instances = ["I had a pleasure to work with such dedicated team. Looking forward to cooperate with each and every one of them again."]
result = server.test("/v2/models/mymodel/infer", {"instances": instances})
result

[2]


## Example 2 - Misleading Positive Sentiment

In [10]:
instances = [
    'This app is amazingly.',
    'As much as I hate to admit it, the new added feature is surprisingly user friendly.'
]
result2 = server.test("/v2/models/mymodel/infer", {"instances": instances})
result2

[0, 2]


## Remote Deployment
Create a function object with custom specification.

In [ ]:
fn.deploy(project='bert-example')